In [92]:
class Args:
    pass
args = Args()
setattr(args, "path_to_model_config", "/home/jano1906/git/jointformer/configs/models/jointformer")      # Not clear which configs to choose
setattr(args, "path_to_task_config", "/home/jano1906/git/jointformer/configs/tasks/guacamol/physchem")  # Not clear which configs to choose

In [93]:
from jointformer.configs.task import TaskConfig
from jointformer.configs.model import ModelConfig

from jointformer.utils.tokenizers.auto import AutoTokenizer
from jointformer.models.auto import AutoModel
from guacamol.assess_distribution_learning import assess_distribution_learning, DistributionMatchingGenerator

In [94]:
model_config = ModelConfig.from_pretrained(args.path_to_model_config)
task_config = TaskConfig.from_pretrained(args.path_to_task_config)

In [71]:
from jointformer.models.jointformer import Jointformer
model: Jointformer = AutoModel.from_config(model_config) # NOTE: model is not using caching for generation
tokenizer = AutoTokenizer.from_config(task_config)

In [72]:
model = model.cuda()
model = model.eval()

In [77]:
tokenizer # NOTE: tokenizer doesn't expose eos_token_id properties
# NOTE: model does not contain which tokenizer it is compatible with

SmilesTokenizer(name_or_path='', vocab_size=591, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[EOS]', 'pad_token': '[PAD]', 'cls_token': '[BOS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	11: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	14: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	591: AddedToken("[EOS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	592: AddedToken("[BOS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [86]:
output = model.generate(592, 591, 0, 25, 10, device="cuda")

In [87]:
tokenizer.decode(output) # what to do with [UNK] tokens?

['[Te][Zr+3][NH+][Pd+][PH3+][13c][C@@H][K][BrH+][Cd+][Al+2][V][YH][64Cu][Nb][SH5][Y+3][PH4+][Os][PH4][Zr+3][Fe+3][Ag]',
 '[Bi+3]F[At][At][Cr+][BaH][Pt][Pd][PH5][La][N+3][S@@]SS[W+4][Pb][N-3][Rb+]))[P+]1',
 '[V+4][O][unused2][Cs+][Tl+2][2NH][14cH][NH3+2][Cu-2][B+3][13cH][Os][Hg+2][Co+3][Ru+][Er+3][te][AcH][SiH3][Pb+3][Br][SiH3]',
 'Br[14CH3][P+3][N-][N+2][Nb+5][NH2-][Pd-][Ca+][Zn][Ni-][Sb+3]-%21[Pt-2]',
 '[nH][SiH4]\\[Al-][Be+2][Be+2][SH3+][Pd][B+3][Au+3][Pd-][Nb][Al-][Al-]',
 '[Cr+3][nH+][unused1][Ho+3]5[Fr][As][V+5][124I-][Rh+2]-[Y+3][N-]2[IrH][Cu+3][Pr][cH+][1H][1H][P@@H][32P][Be]',
 '[SH5][Ni+3][TaH3][PH+][Yb+2][Cd+][S-][Ni-][c-]%24[14CH3][CuH2-][14C@H]3[Pb+4][SH2][Cr][14CH3][118Sn]([Dy][Li+]s',
 '[Ho]([AsH2][AlH2+][Cu+3][SbH2][Ca+2][N@@][V+3][SiH4+][SiH4+][Sb][Nb+3][Nd][Nd][NH4+][AsH+][PH-][ArH][Y+3][Br+2][NH+3]',
 '[14C@H][13cH][Yb+2][V]%14[13C][LaH][GeH2][CH2-][Nd+]%21[Pd][15OH2][18O][18O][Re][Th+4][Y][Al+3][Sc][P@@][Mo+2][Cs]',
 '[Cd+][Sm+3][Os][Os][18O][Sb+5][Ag+2][CH3+][Sn+3]%

In [91]:
from typing import List


class DistributionMatchingGeneratorWrapper(DistributionMatchingGenerator):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def generate(self, number_samples: int) -> List[str]:
        return model.generate(592, 591, 0, 25, number_samples, device="cuda") #unclear how long sequences to generate

In [90]:
assess_distribution_learning(DistributionMatchingGeneratorWrapper(model), "/home/jano1906/datasets/guacamol/guacamol_v1_train.smiles")

KeyboardInterrupt: 